In [1]:
import xarray as xr
import numpy as np
import pandas as pd

In [2]:
local_path = '/Users/fabienaugsburger/myCloud/Unil/Master/Thesis/Datasets/Curnagl/ERA5_1990_2m_dewpoint_temperature.nc'

dew_point_xr = xr.open_dataset(local_path)
dew_point_xr

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 721, time: 2920)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 1990-01-01 ... 1990-12-31T21:00:00
Data variables:
    d2m        (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-10-13 21:02:30 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [3]:
# import 1st track

#head = ['lon_east', 'lon_west', 'lat_south', 'lat_north']
#track_1 = pd.read_csv('tc_irad_tracks/tc_irad_1.txt')
#track_1.columns = head

# import all tracks

for i in range(0,4):
    locals()['track_' + str(i+1)] = pd.read_csv('tc_irad_tracks/tc_irad_' + str(i+1) + '.txt')
    

In [4]:
# slice the time dimension

dates = pd.read_csv('storms_start_end.csv', sep=',')

'''
start_first = dates['start_date'][0]
end_first = dates['end_date'][0]
'''
for i in range(0,4):
    start_temp = dates['start_date'][i]
    end_temp = dates['end_date'][i]

    # Create a new dataset for each iteration
    new_dataset = xr.Dataset({
        'd2m': dew_point_xr['d2m'],
        # Add other variables as needed
    })

    # Optionally, you can update the time dimension for the new dataset
    new_dataset = new_dataset.sel(time=slice(start_temp, end_temp))

    # Dynamically name the dataset variable
    locals()[f"dew_point_xr_{i}"] = new_dataset

'''
dew_point_xr = dew_point_xr.sel(time=slice(start_first, end_first))
dew_point_xr
'''

'\ndew_point_xr = dew_point_xr.sel(time=slice(start_first, end_first))\ndew_point_xr\n'

In [5]:
dew_point_xr_0

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 721, time: 62)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 1990-01-21T03:00:00 ... 1990-01-28T18:00:00
Data variables:
    d2m        (time, latitude, longitude) float32 ...

In [6]:
# create new pd dataframe
'''
first_slice = track_1.iloc[0]

test_dp     = dew_point_xr.sel(longitude=slice(first_slice['lon_east'], first_slice['lon_west']), latitude=slice(first_slice['lat_north'], first_slice['lat_south']))
test_dp'''


"\nfirst_slice = track_1.iloc[0]\n\ntest_dp     = dew_point_xr.sel(longitude=slice(first_slice['lon_east'], first_slice['lon_west']), latitude=slice(first_slice['lat_north'], first_slice['lat_south']))\ntest_dp"

In [7]:
# plot the first slice with matplotlib
'''
import matplotlib.pyplot as plt


plt.figure(figsize=(10, 5))
plt.pcolormesh(test_dp.longitude, test_dp.latitude, test_dp.d2m[0,:,:], cmap='coolwarm')'''

"\nimport matplotlib.pyplot as plt\n\n\nplt.figure(figsize=(10, 5))\nplt.pcolormesh(test_dp.longitude, test_dp.latitude, test_dp.d2m[0,:,:], cmap='coolwarm')"

In [8]:
# check if it's correct
'''
lon_test = np.asanyarray(dew_point_xr.longitude[:])
lat_test = np.asanyarray(dew_point_xr.latitude[:])


closest_lon_w =np.abs(lon_test - first_slice['lon_west']).argmin()
closest_lon_e = np.abs(lon_test - first_slice['lon_east']).argmin()
closest_lat_s = np.abs(lat_test - first_slice['lat_south']).argmin()
closest_lat_n = np.abs(lat_test - first_slice['lat_north']).argmin()

lon_test[closest_lon_w]
lat_test[closest_lat_s]
'''
# works

"\nlon_test = np.asanyarray(dew_point_xr.longitude[:])\nlat_test = np.asanyarray(dew_point_xr.latitude[:])\n\n\nclosest_lon_w =np.abs(lon_test - first_slice['lon_west']).argmin()\nclosest_lon_e = np.abs(lon_test - first_slice['lon_east']).argmin()\nclosest_lat_s = np.abs(lat_test - first_slice['lat_south']).argmin()\nclosest_lat_n = np.abs(lat_test - first_slice['lat_north']).argmin()\n\nlon_test[closest_lon_w]\nlat_test[closest_lat_s]\n"

In [9]:
#slice with new boundaries
'''
plt.pcolormesh(np.flipud(dew_point_xr['d2m'][0,176:208,1157:1189]), cmap='coolwarm')
plt.colorbar()'''

"\nplt.pcolormesh(np.flipud(dew_point_xr['d2m'][0,176:208,1157:1189]), cmap='coolwarm')\nplt.colorbar()"

In [10]:
lat_test[208]

NameError: name 'lat_test' is not defined

In [11]:
# iterate through each time step

var_out = []
'''
for i in range(0, len(track_1)):
    lon_e_temp, lon_w_temp, lat_s_temp, lat_n_temp = track_1.iloc[i]

    lon_test = np.asanyarray(dew_point_xr.longitude[:])
    lat_test = np.asanyarray(dew_point_xr.latitude[:])

    closest_lon_w =np.abs(lon_test - lon_w_temp).argmin()
    closest_lon_e = np.abs(lon_test - lon_e_temp).argmin()
    closest_lat_s = np.abs(lat_test - lat_s_temp).argmin()
    closest_lat_n = np.abs(lat_test - lat_n_temp).argmin()

    temp_ds = np.asarray(dew_point_xr['d2m'])

    var_out.append(temp_ds[i,closest_lat_n:closest_lat_s,closest_lon_e:closest_lon_w])
'''
var_out
for j in range (0,4):
    track_temp = pd.read_csv(f'tc_irad_tracks/tc_irad_{j+1}.txt')
    dew_point_temp = locals()[f"dew_point_xr_{j}"]
    var_out_temp = []
    for i in range(0, len(track_temp)):
        lon_e_temp, lon_w_temp, lat_s_temp, lat_n_temp = track_temp.iloc[i]

        lon_test = np.asanyarray(dew_point_temp.longitude[:])
        lat_test = np.asanyarray(dew_point_temp.latitude[:])

        closest_lon_w = np.abs(lon_test - lon_w_temp).argmin()
        closest_lon_e = np.abs(lon_test - lon_e_temp).argmin()
        closest_lat_s = np.abs(lat_test - lat_s_temp).argmin()
        closest_lat_n = np.abs(lat_test - lat_n_temp).argmin()

        # Use .roll to handle the 0°/360° boundary

        if dew_point_temp.longitude[closest_lon_w] < 100 and dew_point_temp.longitude[closest_lon_e] > 100:
            roll_shift = {'longitude':closest_lon_w, 'longitude': closest_lon_e}
            dew_point_temp_rolled = dew_point_temp.roll(roll_shift, roll_coords=True)
        else:
            dew_point_temp_rolled = dew_point_temp
        # Slice the dataset based on the rolled longitudes and latitudes
        temp_ds = dew_point_temp_rolled['d2m'].isel(time=i, latitude=slice(closest_lat_n, closest_lat_s),
                                                    longitude=slice(closest_lon_e, closest_lon_w)).values

        #temp_ds = np.asarray(dew_point_temp['d2m'])

        #var_out_temp.append(temp_ds[i,closest_lat_n:closest_lat_s,closest_lon_e:closest_lon_w])
        var_out_temp.append(temp_ds)

    locals()[f"var_out_{j+1}"] = var_out_temp


In [12]:
# test

'''for j in range(0, 4):
    track_temp = pd.read_csv(f'tc_irad_tracks/tc_irad_{j+1}.txt')
    dew_point_temp = locals()[f"dew_point_xr_{j}"]
    var_out_temp = []

    for i in range(0, len(track_temp)):
        lon_e_temp, lon_w_temp, lat_s_temp, lat_n_temp = track_temp.iloc[i]

        lon_test = np.asanyarray(dew_point_temp.longitude[:])
        lat_test = np.asanyarray(dew_point_temp.latitude[:])

        closest_lon_w = np.abs(lon_test - lon_w_temp).argmin()
        closest_lon_e = np.abs(lon_test - lon_e_temp).argmin()
        closest_lat_s = np.abs(lat_test - lat_s_temp).argmin()
        closest_lat_n = np.abs(lat_test - lat_n_temp).argmin()

        # Check if the indices create a valid slice
        if closest_lat_s > closest_lat_n and closest_lon_w < closest_lon_e:
            # Use .roll to handle the 0°/360° boundary
            roll_shift = {'longitude': closest_lon_e, 'longitude': closest_lon_w}
            dew_point_temp_rolled = dew_point_temp.roll(roll_shift, roll_coords=True)

            # Slice the dataset based on the rolled longitudes and latitudes
            temp_ds = dew_point_temp_rolled['d2m'].isel(time=i, latitude=slice(closest_lat_n, closest_lat_s),
                                                        longitude=slice(closest_lon_e, closest_lon_w)).values

            var_out_temp.append(temp_ds)
        else:
            print(f"Invalid indices for square at time index {i} in track {j+1}")

    locals()[f"var_out_{j+1}"] = var_out_temp
'''

roll_test = {'longitude': 1400, 'longitude': 20}

test_ds_rolled = dew_point_xr.roll(roll_test, roll_coords=True)

test_ds_rolled

test_ds = test_ds_rolled['d2m'].isel(latitude=slice(100, 140), longitude=slice(1400, 20))
test_ds

<xarray.DataArray 'd2m' (time: 2920, latitude: 40, longitude: 0)>
array([], shape=(2920, 40, 0), dtype=float32)
Coordinates:
  * longitude  (longitude) float32 
  * latitude   (latitude) float32 65.0 64.75 64.5 64.25 ... 55.75 55.5 55.25
  * time       (time) datetime64[ns] 1990-01-01 ... 1990-12-31T21:00:00
Attributes:
    units:      K
    long_name:  2 metre dewpoint temperature

In [28]:
#print(closest_lon_w, closest_lon_e, closest_lat_s, closest_lat_n)
temp_ds

array([], shape=(32, 0), dtype=float32)

In [20]:
# check if each square has the same shape

for i in range(0, len(var_out_3)):
    print(var_out_3[i].shape)

(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 0)
(32, 1408)
(32, 1408)
(32, 1408)
(32, 1408)
(32, 1408)
(32, 1408)
(32, 1408)
(32, 1408)
(32, 1408)
(32, 1408)
(32, 1408)
(32, 1408)
(32, 1408)
(32, 0)
(32, 0)
(32, 0)
(32, 0)


In [71]:
var_out_4[15]

array([], shape=(32, 0), dtype=float32)

In [13]:
# mean, min, max value of each time step

mean_out_d2m = []
min_out_d2m = []
max_out_d2m = []

for i in range(0, len(var_out)):
    mean_out_d2m.append(np.mean(var_out[i]))
    min_out_d2m.append(np.min(var_out[i]))
    max_out_d2m.append(np.max(var_out[i]))

In [14]:
# save as csv

df_mean = pd.DataFrame(mean_out_d2m)
df_min = pd.DataFrame(min_out_d2m)
df_max = pd.DataFrame(max_out_d2m)

df_mean.to_csv('d2m_mean.csv')
df_min.to_csv('d2m_min.csv')
df_max.to_csv('d2m_max.csv')

For all variables, (class by variable (folder), then by storm)) 